In [110]:
# pip install tqdm
# !pip install -U scikit-learn
# !pip install seaborn

In [111]:
import warnings
import requests
import urllib.request
from tqdm import tqdm
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import math

warnings.filterwarnings(action='ignore')
gameNum = 10000


# 데이터 수집
- ## 저장된 LOL 데이터 및 LOL API 의 챔피언, 아이템 정보 수집

In [112]:
data = []
iter = 0
for line in tqdm(open('lolData', 'r')):
    data.append(json.loads(line))
    iter+=1
    if iter==gameNum:
        break

9999it [00:05, 1883.12it/s]


In [113]:
data[0].keys()

dict_keys(['seasonid', 'queueid', 'gameid', 'participantidentities', 'gameversion', 'platformid', 'gamemode', 'mapid', 'gametype', 'teams', 'participants', 'gameduration', 'gamecreation', 'participantextendedstats'])

In [114]:
data[0]['participantidentities'][0]['player']['summonerid']

'PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA'

In [115]:
data[0]['gamecreation']

1599158094

In [116]:
# 아이템 정보수집
itemURL = 'http://ddragon.leagueoflegends.com/cdn/10.18.1/data/en_US/item.json'
request = requests.get(itemURL)
itemData = request.json()

In [117]:
itemDict = {}
for i_name in itemData['data'].keys() :
    itemDict[int(i_name)]=itemData['data'][i_name]['name']

In [118]:
itemDict[3123]

"Executioner's Calling"

In [119]:
# 챔피언 정보 수집
championURL = 'http://ddragon.leagueoflegends.com/cdn/10.23.1/data/en_US/champion.json'
request = requests.get(championURL)

In [120]:
championData = request.json()
request.json().keys()

dict_keys(['type', 'format', 'version', 'data'])

In [121]:
championDict = {}
for c_name in championData['data'].keys() :
    championDict[int(championData['data'][c_name]['key'])]=c_name

In [122]:
championDict[58]

'Renekton'

In [123]:
data[0]['participants'][0]['stats'].keys()

dict_keys(['physicaldamagedealt', 'neutralminionskilledteamjungle', 'magicdamagedealt', 'totalplayerscore', 'deaths', 'win', 'neutralminionskilledenemyjungle', 'largestcriticalstrike', 'totaldamagedealt', 'magicdamagedealttochampions', 'visionwardsboughtingame', 'damagedealttoobjectives', 'largestkillingspree', 'item1', 'quadrakills', 'totaltimecrowdcontroldealt', 'wardskilled', 'firsttowerassist', 'firsttowerkill', 'item2', 'item3', 'item0', 'firstbloodassist', 'visionscore', 'wardsplaced', 'item4', 'item5', 'item6', 'turretkills', 'triplekills', 'damageselfmitigated', 'champlevel', 'goldearned', 'magicaldamagetaken', 'kills', 'doublekills', 'truedamagetaken', 'assists', 'unrealkills', 'neutralminionskilled', 'objectiveplayerscore', 'combatplayerscore', 'damagedealttoturrets', 'physicaldamagedealttochampions', 'goldspent', 'truedamagedealt', 'truedamagedealttochampions', 'participantid', 'pentakills', 'totalheal', 'totalminionskilled', 'firstbloodkill', 'largestmultikill', 'sightwards

# 데이터 가공
- ## time으로 나누어 연속 피쳐로 생성 
- ## 표준 정규화 진행

In [124]:
assist = []
kill = []
death = []
kda = []
gold = []
level = []
champion_num = []
summoner_id = []

items = [ [] for _ in range(7)]

for gameCnt in tqdm(range(gameNum)) : 
    time = data[gameCnt]['gameduration'] / 60
    for i in range(10) :
        champion_num.append(data[gameCnt]['participants'][i]['championid'])
        assist.append(data[gameCnt]['participants'][i]['stats']['assists'] /time )
        kill.append(data[gameCnt]['participants'][i]['stats']['kills'] /time )
        death.append(data[gameCnt]['participants'][i]['stats']['deaths'] /time )
        gold.append(data[gameCnt]['participants'][i]['stats']['goldearned'] / time)
        level.append(data[gameCnt]['participants'][i]['stats']['champlevel'] / time)
        summoner_id.append(data[gameCnt]['participantidentities'][i]['player']['summonerid'])

        for item_num in range(7) :
            items[item_num].append(data[gameCnt]['participants'][i]['stats']['item'+str(item_num)])
        
        if death[len(death)-1] == 0 :
            kda.append(1.5 * (assist[len(assist)-1] + kill[len(kill)-1]))
        else :
            kda.append((assist[len(assist)-1] + kill[len(kill)-1]) / death[len(death)-1])


assistCnt = [0 for _ in range(int(max(assist)*100)+2)]
killCnt = [0 for _ in range(int(max(kill)*100)+2)]
deathCnt = [0 for _ in range(int(max(death)*100)+2)]
kdaCnt = [0 for _ in range(int(max(kda)*10)+2)]
goldCnt = [0 for _ in range(int(max(gold))+2)]

100%|██████████| 10000/10000 [00:00<00:00, 15195.72it/s]


In [125]:
for item in range(len(assist)) :
    assistCnt[int(assist[item] * 100)] +=1
    killCnt[int(kill[item] * 100)] +=1
    deathCnt[int(death[item] * 100)] +=1
    kdaCnt[int(kda[item] * 10)] +=1
    goldCnt[int(gold[item])] +=1


# 데이터 시각화
- ## 표준정규분포를 따르는지 그래프를 통해 확인
- ## 피쳐별 분포정도를 Box Plot 를 통해 확인

In [126]:
# from matplotlib import pyplot as plt
# import numpy as np

# x_axis = np.arange(len(killCnt))
# plt.bar(x_axis, killCnt)

In [127]:
# x_axis = np.arange(len(goldCnt))
# plt.bar(x_axis, goldCnt)

In [128]:
kill_df = pd.DataFrame(kill)
death_df = pd.DataFrame(death)
x_data = pd.DataFrame()
x_data['kill'] = kill
x_data['assist'] = assist
x_data['death'] = death
x_data['gold'] = gold
x_data['level'] = level

x_data['troll'] = 0
x_data['championid'] = champion_num
x_data['summoner_id'] = summoner_id

x_data

,kill,assist,death,gold,level,troll,championid,summoner_id
0,0.158520,0.356671,0.118890,359.167768,0.515192,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA
1,0.118890,0.475561,0.277411,278.678996,0.435931,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y
2,0.317041,0.158520,0.237781,423.725231,0.515192,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy
3,0.079260,0.277411,0.356671,323.659181,0.475561,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA
4,0.356671,0.198151,0.118890,427.093791,0.515192,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ
...,...,...,...,...,...,...,...,...
99995,0.183374,0.293399,0.183374,496.613692,0.550122,0,236,XU_3wTozp_g-sec3SdeBp7WOg61BU4Bsi9P636axectzPRY
99996,0.330073,0.256724,0.183374,437.090465,0.550122,0,245,oYgrmGD1cGqAN6My_talEYVTEp9YpZkjDsIZ3NMhjWqI9g
99997,0.036675,0.513447,0.220049,242.603912,0.440098,0,432,q4uKpVa1RT8cihUV3Wa1t9Y7o-9CttHF3mOYxtdkksSUKQ
99998,0.403423,0.220049,0.220049,416.625917,0.513447,0,55,Q_fw1SGj4BUHxM3gFnRWx7WcchwUTZbN4S-F7W0jwguxi-4


In [129]:
# feature_list = ['kill','assist','death','gold','level']

# for feature in feature_list :
#     plot_x_data = [x_data[feature][x_data['troll']==0],x_data[feature][x_data['troll']==1]]

#     if feature == 'gold' :
#         plt.yticks([-10 + 1 * i for i in range(16)])
#     else :
#         plt.yticks([-10 + 1 * i for i in range(16)])
    
#     flierprops = dict(markerfacecolor='r',markersize=0.1)
#     plt.title(feature)
#     plt.grid()

#     plt.boxplot(plot_x_data, flierprops=flierprops)
#     plt.xticks([1,2] , ['not_afk','afk'])

#     plt.show()


# 챔피언별 피쳐들로 표준 정규화 진행

In [130]:
#챔피언별 피쳐들로 표준 정규화 진행

features = ['kill','assist','death','gold','level']

champion_list = x_data['championid'].unique()

for cham_id in tqdm(champion_list) : 
    for f in features : 
        f_mean = x_data[f][x_data['championid']==cham_id].mean()
        f_std = x_data[f][x_data['championid']==cham_id].std() + math.pow(10,-8)

        x_data[f][x_data['championid']==cham_id] -= f_mean
        x_data[f][x_data['championid']==cham_id] /= f_std

x_data

100%|██████████| 150/150 [00:02<00:00, 58.01it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id
0,-0.343376,1.745007,-0.491769,-0.431260,-0.512853,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA
1,1.180590,0.130797,1.246795,0.398971,-0.230212,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y
2,0.929154,-0.218189,0.525580,0.450369,-0.496110,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy
3,-1.067484,0.378228,1.735096,-0.896303,-0.665233,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA
4,0.802854,-0.087691,-0.540774,0.072379,0.090969,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ
...,...,...,...,...,...,...,...,...
99995,-0.181216,0.817749,-0.076838,0.950896,0.118960,0,236,XU_3wTozp_g-sec3SdeBp7WOg61BU4Bsi9P636axectzPRY
99996,0.599617,0.275636,0.099153,0.543064,0.153125,0,245,oYgrmGD1cGqAN6My_talEYVTEp9YpZkjDsIZ3NMhjWqI9g
99997,-0.517921,0.477724,0.607669,-0.546329,-0.406754,0,432,q4uKpVa1RT8cihUV3Wa1t9Y7o-9CttHF3mOYxtdkksSUKQ
99998,0.642316,0.283726,0.252854,0.130415,-0.408584,0,55,Q_fw1SGj4BUHxM3gFnRWx7WcchwUTZbN4S-F7W0jwguxi-4


In [131]:
itemStr=['item' + str(num) for num in range(6)]

In [132]:
#item Task

total_item_dict = {}
total_item_number_dict = {}

for gameCnt in tqdm(range(gameNum)) : 
    for i in range(10) :
        summoner_itemDict = {}
        for it in itemStr :
            if summoner_itemDict.get(data[gameCnt]['participants'][i]['stats'][it])==None :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] = 1
            else :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] += 1
        
        for itemNum in summoner_itemDict.keys() :
            if summoner_itemDict[itemNum]>=2 :
                if itemNum!=0:
                    if total_item_dict.get(itemDict[itemNum]) == None :
                        total_item_dict[itemDict[itemNum]] =1
                        total_item_number_dict[itemDict[itemNum]] = {2:0,3:0,4:0,5:0,6:0}
                        total_item_number_dict[itemDict[itemNum]][summoner_itemDict[itemNum]]+=1

                    else : 
                        total_item_dict[itemDict[itemNum]] +=1
                        total_item_number_dict[itemDict[itemNum]][summoner_itemDict[itemNum]]+=1


list(total_item_dict)[:10]


100%|██████████| 10000/10000 [00:00<00:00, 34385.63it/s]


["Doran's Blade",
 "Executioner's Calling",
 'Faerie Charm',
 "Doran's Ring",
 'Stopwatch',
 'Broken Stopwatch',
 'Long Sword',
 'Rejuvenation Bead',
 'Cloth Armor',
 'Amplifying Tome']

# 아이템을 이용하여 트롤 유저 탐색과정 진행
- ## 아이템 중복구매 유저를 탐색
- ## 탐색한 아이템 중복구매 유저를 통해 트롤 유저 라벨링 진행

In [133]:
overTwo = [2421,2424,2420]
          # 깨진 초시계 2개 , 초시계
overSix = [1006         , 1004      ,1027       ,  1028     , 1082]
            #원기회복구슬 ,요정의부적,사파이어 수정,루비 수정, 암흑의 인장
'''
for i in itemDict:
    if itemDict[i] == 'Rejuvenation Bead' :
        print(i)
'''     

"\nfor i in itemDict:\n    if itemDict[i] == 'Rejuvenation Bead' :\n        print(i)\n"

In [134]:
item_six_number = []
item_six_cnt = []
for key,value in sorted(total_item_dict.items() , key = lambda x : total_item_number_dict[x[0]][6] , reverse = True) :
    #print(key,value,total_item_number_dict[key][2],total_item_number_dict[key][3],total_item_number_dict[key][4],total_item_number_dict[key][5],total_item_number_dict[key][6])
    print(key,total_item_number_dict[key][6])

Rejuvenation Bead 214
Faerie Charm 82
Broken Stopwatch 46
Dark Seal 36
Sapphire Crystal 34
Stopwatch 19
Ruby Crystal 18
Doran's Ring 13
Kindlegem 13
Dagger 12
Chalice of Harmony 12
Cull 10
Cloth Armor 9
Bami's Cinder 9
Doran's Blade 8
Long Sword 7
Crystalline Bracer 7
Amplifying Tome 6
Kircheis Shard 6
Executioner's Calling 3
Needlessly Large Rod 3
Glacial Shroud 3
B. F. Sword 3
Doran's Shield 3
Aether Wisp 3
Catalyst of Aeons 2
Null-Magic Mantle 2
Negatron Cloak 2
Vampiric Scepter 2
Giant's Belt 1
Cloak of Agility 1
Serrated Dirk 1
Mejai's Soulstealer 1
Pickaxe 1
Spectre's Cowl 1
Quicksilver Sash 1
Forbidden Idol 1
Fiendish Codex 1
Chain Vest 1
Elixir of Wrath 1
Rabadon's Deathcap 0
Zeal 0
Recurve Bow 0
Guardian Angel 0
Redemption 0
Sterak's Gage 0
Blasting Wand 0
Jaurim's Fist 0
Stinger 0
Infinity Edge 0
Void Staff 0
Caulfield's Warhammer 0
Duskblade of Draktharr 0
Lost Chapter 0
Hextech Revolver 0
Aegis of the Legion 0
Death's Dance 0
Statikk Shiv 0
Umbral Glaive 0
Ardent Censer 0
R

In [135]:
# 각 아이템 구매한 유저들 번호 
troll_playerList = []
for gameCnt in tqdm(range(gameNum)) : 
    for i in range(10) :
        summoner_itemDict = {}
        for it in itemStr :
            if summoner_itemDict.get(data[gameCnt]['participants'][i]['stats'][it])==None :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] = 1
            else :
                summoner_itemDict[data[gameCnt]['participants'][i]['stats'][it]] += 1
        
        for itemNum in summoner_itemDict.keys() :
            if (itemNum in overTwo and summoner_itemDict[itemNum]>=2  ) or (itemNum in overSix and summoner_itemDict[itemNum]==6):
                troll_playerList.append([gameCnt,i])

troll_playerList[:10]

100%|██████████| 10000/10000 [00:00<00:00, 31264.26it/s]


[[6, 0],
 [13, 9],
 [22, 9],
 [30, 4],
 [35, 0],
 [37, 8],
 [43, 6],
 [56, 1],
 [62, 5],
 [82, 1]]

In [136]:
len(troll_playerList)

531

In [137]:
#dictionary에 troll_player들 넣어서 각 유저별 아이템 판매횟수 확인
troll_player_dict = {}

for game_num,player_cnt in troll_playerList :
    if troll_player_dict.get(data[game_num]['participantidentities'][player_cnt]['player']['summonerid']) == None :
        troll_player_dict[data[game_num]['participantidentities'][player_cnt]['player']['summonerid']] = 1
    else :
        troll_player_dict[data[game_num]['participantidentities'][player_cnt]['player']['summonerid']] += 1

troll_player_dict_sorted = sorted(troll_player_dict.items(),key = lambda x : x[1], reverse=True)

troll_player_dict_sorted[:20]

[('Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM', 8),
 ('2zz5blkjYNGp3ypIf2Rv7hlmgaFP0Lyfur8Ukzujua4zDQ', 4),
 ('51CMgUaia2ZlTo8TkFv16pMN_TMXUF6ww8xDnJl60t8mJuo', 3),
 ('3lZaLVQn3BXZIMrWRF2iWerxroM2l2A68XzMnZlKEF6p72k', 3),
 ('T6kOMf_mNkBLI9Xp9jyDOOCk3BNABZFVAPHbEf2ePIcvLI0', 3),
 ('0M-qX85hegHj_x4YcMHTMH8qOFoD1LzCF5SMfIbCbJKWXtEk', 3),
 ('RSqzwOlH-hgS7kM1cUC9Uwhkt6VbZgcRT1Ei7FA-LBgdSg', 3),
 ('VrPo9rsyWtihufnWTwXEF9mMAhzj2qbDvF0FPiZZ0WamQL8', 2),
 ('UN4OxxeTlRfhZVi6Qs6gxindmS_kb6Ma8GgxsQcd7DAoOWg', 2),
 ('5xkIxvJR1mlGKIdoRjLDvo17jj_RP9Cl_BR7yo0wvd-LnMU', 2),
 ('mOsGKBOaxsQQx0EPiFJxQxHOu2f6I4iJgRp1dTUCfh6k4Ahg', 2),
 ('DYCswFZRuoMff954RSs2ZoN5NPKWUt90pYrDGZGEvxdkgk0', 2),
 ('kW9Fg7SpHhVH2DTxf2alC8yx2Ks4sRipqJtaSIkj5CoVGN0', 2),
 ('ZDGZuk8A8eEkqfoumh4Sgm4VXl-VixXbl7ArZm83S67cB4A', 2),
 ('RxDcfUobbW1sTSfwhg9qc2RF-7kSRMUe9EhgFcgGeW8HHEw', 2),
 ('2qzOcd22gAk-zwBVPJcEX8JIAsvnIYMldyapQfS8ptL9sMI', 2),
 ('ymlzljXNA_w9OmCAlHg1Ns75s0phoHd1impHcs1nNEdjDN5y', 2),
 ('0SVWxrhpRSsjgaR9h6ygZgfqpDD

In [138]:
for i,item_num in enumerate(itemStr) : 
    x_data[item_num] = items[i]

# 트롤 유저 탐색
- ## 트롤 유저가 보유한 아이템 구매 현황을 HTML 을 이용하여 시각화

In [139]:
from IPython.core.display import HTML

most_troll_player_id = troll_player_dict_sorted[0][0]

df_shuffled_image = x_data[x_data['summoner_id'] == most_troll_player_id]

for item_num in tqdm(itemStr) :
    df_shuffled_image[item_num] = df_shuffled_image[item_num].astype(str)
    df_shuffled_image[item_num] = '<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/' + df_shuffled_image[item_num] + '.png\' weight=25 height=25>'

for item_num in tqdm(itemStr) :
    for i,item_img in enumerate(df_shuffled_image[item_num]) :
        if item_img == "<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/0.png\' weight=25 height=25>" :
            df_shuffled_image[item_num][df_shuffled_image.index[i]] = ''

df_shuffled_image_html=df_shuffled_image.to_html(escape=False)
#print(df_shuffled_image_html[:1600])
# HTML(df_shuffled_image_html)


100%|██████████| 6/6 [00:00<00:00, 4332.21it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
7515,2.173647,1.767286,-1.350843,2.678697,1.707086,0,82,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
8295,-0.955169,0.709283,-0.965989,0.282145,0.967431,0,350,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
34890,-1.101918,-1.588167,-0.964603,-2.027894,-3.399348,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
39785,0.589583,-1.515974,2.490203,-0.290494,-0.423415,0,235,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
40275,-1.392273,-1.683863,0.361523,-3.352880,-6.246304,0,51,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
43938,-0.231920,-0.731629,-0.074163,-1.208704,-1.738174,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
50828,-1.361806,-1.588167,-0.806331,-2.732676,-3.832546,0,81,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
52933,-1.164665,-1.001022,-0.632536,-1.621782,-1.183154,0,235,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
64090,-0.214889,0.430128,-1.451541,0.325548,0.549773,0,350,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,
69787,-0.179052,1.058447,-1.732681,0.763133,0.056198,0,103,Dw-MTTbD78UU4xosHihNXSRXgt4nUqfHz-FA7KjutKp1BoM,,,,,,


In [140]:
print(x_data[x_data['summoner_id'] == most_troll_player_id]['kill'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['death'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['assist'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['gold'].mean(),x_data[x_data['summoner_id'] == most_troll_player_id]['level'].mean())

-0.38384616056855597 -0.5126960483286783 -0.4143678177120142 -0.7184906737014559 -1.354245205461863


In [141]:
len(troll_player_dict)

487

In [142]:
pd.set_option('display.max_colwidth', 500)

# Data를 ML 및 PCA 과정을 진행하기 좋도록 가공
- ## 위의 아이템을 통해 Labeling 한 Troll 유저 데이터를 기존의 데이터와 결합하여 X와 Y, 즉, Feature와 Target으로 생성

In [143]:
for i in tqdm(range(len(troll_playerList))) :
    x_data['troll'][troll_playerList[i][0]*10 + troll_playerList[i][1]] = 1
x_data

100%|██████████| 531/531 [00:00<00:00, 8799.07it/s]


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
0,-0.343376,1.745007,-0.491769,-0.431260,-0.512853,0,38,PF7wuKbEjIH7bUEivDRsJbiah9EF9EzRPO-LFLucLai7avA,1054,3003,2421,3020,3027,0
1,1.180590,0.130797,1.246795,0.398971,-0.230212,0,497,JRuCiFpuzcsnQ9Gadn5A_0bm66-PrFaA3WKP5juhgetrx3Y,2065,3109,3859,0,0,3117
2,0.929154,-0.218189,0.525580,0.450369,-0.496110,0,164,o5ZjlQkLgDZzDG_dkT0GVL7X34kp51abqkUN6P7K2WPeY1dy,1055,3105,3078,3158,1036,3074
3,-1.067484,0.378228,1.735096,-0.896303,-0.665233,0,19,TUdiT-gEHdtkmr9JkhqvtNy5L_nutzsw2oH23kgO71NSsA,1413,3748,2055,1028,3009,1029
4,0.802854,-0.087691,-0.540774,0.072379,0.090969,0,145,qmcY1AJTdznK_aTgW2e1qwCY1uwbam9HU09Wsa7nI9Wt5sYQ,1055,1052,3134,3508,3006,3115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.181216,0.817749,-0.076838,0.950896,0.118960,0,236,XU_3wTozp_g-sec3SdeBp7WOg61BU4Bsi9P636axectzPRY,3508,3153,1018,3047,3071,1055
99996,0.599617,0.275636,0.099153,0.543064,0.153125,0,245,oYgrmGD1cGqAN6My_talEYVTEp9YpZkjDsIZ3NMhjWqI9g,2420,3108,3152,1402,3111,3100
99997,-0.517921,0.477724,0.607669,-0.546329,-0.406754,0,432,q4uKpVa1RT8cihUV3Wa1t9Y7o-9CttHF3mOYxtdkksSUKQ,3222,2031,3860,2055,3117,3114
99998,0.642316,0.283726,0.252854,0.130415,-0.408584,0,55,Q_fw1SGj4BUHxM3gFnRWx7WcchwUTZbN4S-F7W0jwguxi-4,1054,3146,3157,3020,3165,0


In [144]:
troll_df = x_data[x_data['troll']==1]
troll_df

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
60,-0.479828,-0.917854,1.412417,-0.574662,-0.706747,1,22,zvgRBByyRKJi5Q1i49N2mcnNSr5IiWKxS5xakwE0_CZqKIs,2420,2420,2420,2420,2420,2420
139,-1.018040,-1.324337,2.021819,-1.755397,-0.642089,1,39,6Uz9KiY6ayRRCwhbA6Wc-prEFUfSSAhAmpevAGDprLm6sng,2421,2421,2421,2421,0,0
229,-1.441009,-1.507715,0.255908,-2.357340,-1.128540,1,2,p109-T2B_cR4sG_fzAdfyQlj_nWLL5wCCOnSJ00YJEzEP7oR,1006,1006,1006,1006,1006,1006
304,-0.775539,-0.158237,0.601271,-0.714910,-0.443196,1,60,VrPo9rsyWtihufnWTwXEF9mMAhzj2qbDvF0FPiZZ0WamQL8,1006,1006,1006,1006,1006,1006
350,-0.597536,-0.318745,0.753871,-0.112047,-0.159208,1,875,t8iVT_qOY-mxiRK72VdPxrF-eL8MnCRqc05gZDBYLf2ZzQ,1006,1006,1006,1006,1006,1006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98289,-0.809537,-1.372979,-0.090100,-1.476752,-0.632488,1,107,OXnanspWIjLO4eRB7wzGOuV3yV4sS9noO2pQOV2Xeu1vIw,2421,2421,2421,2421,2421,2421
98385,0.440033,1.526674,0.110552,1.035125,0.013564,1,122,OjpusETLZK7Z6BSbpT38FBP_JY2sV7mGHNnmWKKOSpzsuXY,1006,1006,1006,1006,1006,1006
98470,-0.636244,-1.768627,-1.800921,-2.951538,-6.451224,1,22,6_XA29524HuxGUgzDE0IQhPu5lexfCgo4fcJMBmMSV8jyk8,1006,1006,1006,1006,1006,1006
99127,-0.470773,-0.565796,0.485994,-0.647316,-1.418702,1,875,6l7gSCmbrl0YegJyKKjKJhBkrByT4b_nkNC4AKOdvHa82Fo,2421,2421,2421,2421,2421,2421


In [145]:
print(troll_df['kill'].mean(),troll_df['death'].mean(),troll_df['assist'].mean(),troll_df['gold'].mean(),troll_df['level'].mean())

-0.6564387508068684 0.3578558251856956 -0.8521109148866186 -1.1656433574247882 -0.6583654456609602


In [146]:
len(troll_playerList)

531

In [147]:
#troll_df.drop('time',axis=1,inplace=True)
troll_df['troll'] = 1

In [148]:
sampling_df = x_data.sample(n=len(troll_playerList) * 9, random_state=1)
print(sampling_df['troll'].sum())
sampling_df.drop(sampling_df.loc[sampling_df['troll']==1].index, inplace=True)
print(sampling_df['troll'].sum())
sampling_df

23
0


,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
43660,0.153293,0.834479,-0.670356,1.729769,0.287808,0,43,tqS8ILiI5pIzL5Lyv50QNUfIooxIcjIs727N9YcQv5hSK5o,2055,3107,3853,3158,3504,3174
87278,-1.530838,-1.763910,-1.690318,-2.578429,-2.734205,0,876,_F5YjFqlRR4vRuVdIFpFekoyA_k-2ru8pBrTa7DfFjmcYA,1039,2031,0,0,0,0
14317,-1.059279,0.891256,0.197743,-0.568018,-0.741870,0,80,xZt-WSLNC0kFxBACbtMVcE3_Q5OzIGrJlzDRFZTzoj4RQDk,3857,3155,3117,0,0,3179
81932,-0.522019,-0.503325,2.477672,-0.381065,-0.034252,0,50,q5oFiATi9RqrkZ9TKYSC1T0gYSghHmN4tY_wlgdAsBwY5kk,3853,2055,3157,3020,3136,0
95321,0.444700,-0.247493,1.111254,-0.108176,-1.329817,0,78,TpVBEquSmfUCisO7su1clDNV8Dy9fBA5efPlibdeR31byw,3075,2421,3194,1031,3068,3047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12903,-0.287187,0.219921,-1.941820,0.235511,0.443330,0,30,bQ3VaowvWPqUR8Rz6qG89DhkQXNBMosJ-1iYA7WoUGF6nCyb,1402,2421,3020,3165,3108,0
29305,3.000451,1.797141,0.291098,3.302128,2.875049,0,3,ZcFs7YBjoLKge1g6v38AFRWz3d6jN-kYLWYoa-FhdtgFHSRt,3157,3152,3111,2055,2033,1052
1907,0.170872,1.439800,-0.809354,0.078591,-0.007355,0,80,2Ri8lESkdmvi6Cj2nKVkcbcLMwjjSogp8l-rdjZiHFYPGQ,3857,2055,3044,3067,3179,3117
45984,-0.031674,0.386413,-0.076783,0.426654,0.159068,0,35,lWuBonncIhL3n78-TRNnGvvTnKnr-CmSeKSSrJWDl2KgjMLF,2031,3077,3006,3181,1400,1053


In [149]:
import tensorflow as tf
from tensorflow import keras

sampling_df2 = pd.concat([sampling_df,troll_df])

df_shuffled_total = sampling_df2.sample(frac=1,random_state=1).reset_index(drop=True)
df_shuffled = df_shuffled_total.drop(['championid','summoner_id'],axis=1)
df_shuffled = df_shuffled.drop(itemStr,axis=1)

y = df_shuffled['troll']
#print(df_shuffled['troll'].sum())
#df_shuffled
len(y)

5287

In [150]:
neg, pos = np.bincount(y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 5287
    Positive: 531 (10.04% of total)



In [151]:
x = df_shuffled.drop(['troll'],axis=1)
train_data_len = len(x)*7//10
x_train = x[:train_data_len]
x_test = x[train_data_len:]
y_train = y[:train_data_len]
y_test = y[train_data_len:]

In [152]:
df_shuffled

,kill,assist,death,gold,level,troll
0,-0.434277,-0.618908,-0.113804,-0.705037,-0.705106,0
1,0.307922,-0.152650,-0.829000,0.160268,0.164253,1
2,-0.395869,-0.550199,0.003021,-0.525970,-0.950440,1
3,0.003808,-0.513583,-0.492969,-0.839031,-0.667064,0
4,1.609645,0.382305,-1.451541,2.347640,1.019909,0
...,...,...,...,...,...,...
5282,-0.950406,-0.745653,0.060120,-0.766399,0.272138,0
5283,-0.967382,-1.212238,-0.512353,-1.138467,-1.535250,1
5284,-0.271795,-0.897357,0.186923,-0.773690,-0.634453,0
5285,0.191891,3.128101,0.044998,1.246856,-0.248878,0


In [153]:
from copy import deepcopy
pca_data = df_shuffled.copy()
for i in range(train_data_len,len(df_shuffled)):
    pca_data['troll'].iloc[i] = 0
pca_data

,kill,assist,death,gold,level,troll
0,-0.434277,-0.618908,-0.113804,-0.705037,-0.705106,0
1,0.307922,-0.152650,-0.829000,0.160268,0.164253,1
2,-0.395869,-0.550199,0.003021,-0.525970,-0.950440,1
3,0.003808,-0.513583,-0.492969,-0.839031,-0.667064,0
4,1.609645,0.382305,-1.451541,2.347640,1.019909,0
...,...,...,...,...,...,...
5282,-0.950406,-0.745653,0.060120,-0.766399,0.272138,0
5283,-0.967382,-1.212238,-0.512353,-1.138467,-1.535250,0
5284,-0.271795,-0.897357,0.186923,-0.773690,-0.634453,0
5285,0.191891,3.128101,0.044998,1.246856,-0.248878,0


# PCA 차원축소 및 복원과정을 진행
- ## 축소 할 차원의 수인 n components 수를 1~4 어떤 것을 사용하더라도 큰 차이가 존재하지 않음
- ## 1의 경우가 가장 높았으므로 1의 경우만 기록

In [154]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
reduced = pca.fit_transform(pca_data)
recovered = pca.inverse_transform(reduced)
pca_pred = pd.DataFrame(recovered,columns=pca_data.columns)
pca_pred

,kill,assist,death,gold,level,troll
0,-0.529114,-0.424548,0.328038,-0.687871,-0.448606,0.106095
1,0.257164,0.144703,-0.148439,0.278442,0.193572,0.041523
2,-0.540173,-0.432554,0.334739,-0.701462,-0.457638,0.107003
3,-0.379317,-0.316097,0.237262,-0.503775,-0.326262,0.093793
4,1.615378,1.128026,-0.971504,1.947650,1.302870,-0.070018
...,...,...,...,...,...,...
5282,-0.526796,-0.422869,0.326633,-0.685022,-0.446713,0.105905
5283,-1.003119,-0.767719,0.615280,-1.270410,-0.835742,0.145022
5284,-0.589968,-0.468605,0.364915,-0.762659,-0.498308,0.111093
5285,0.946552,0.643808,-0.566202,1.125681,0.756618,-0.015092


In [155]:
pca_pred['troll'][train_data_len:]

3700    0.052648
3701    0.063705
3702   -0.140490
3703    0.135306
3704    0.143977
          ...   
5282    0.105905
5283    0.145022
5284    0.111093
5285   -0.015092
5286    0.137743
Name: troll, Length: 1587, dtype: float64

## accuracy 보다는 log_loss 를 낮추는 것이 더욱 중요

In [156]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

print('accuracy',accuracy_score(np.round(pca_pred['troll'][train_data_len:]),y_test.astype('float64')))

accuracy 0.8941398865784499


In [157]:
from sklearn.metrics import log_loss
log_loss(y_pred = pca_pred['troll'][train_data_len:], y_true = y_test.astype('float64'))

0.3257376479452615

In [158]:
del data
del x_data

In [159]:
len(x_train)

3700

In [160]:
x_train

,kill,assist,death,gold,level
0,-0.434277,-0.618908,-0.113804,-0.705037,-0.705106
1,0.307922,-0.152650,-0.829000,0.160268,0.164253
2,-0.395869,-0.550199,0.003021,-0.525970,-0.950440
3,0.003808,-0.513583,-0.492969,-0.839031,-0.667064
4,1.609645,0.382305,-1.451541,2.347640,1.019909
...,...,...,...,...,...
3695,0.055473,0.240133,-1.134563,-1.125162,-1.241983
3696,0.506043,-0.298958,-0.546205,1.132147,0.715401
3697,-0.906348,2.406394,-0.310254,0.440409,1.039679
3698,-0.329701,0.353645,-0.548922,-0.871849,-0.802932


# ML Model을 이용하여 예측

In [161]:
model = keras.Sequential()
model.add(tf.keras.layers.Dense(5,input_shape=(5,),activation='relu'))
model.add(tf.keras.layers.Dense(5,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=['binary_accuracy'])

model.fit(x_train,y_train,epochs=20,batch_size=50,verbose=1)

Epoch 1/20
74/74 [==============================] - 0s 347us/step - loss: 0.6660 - binary_accuracy: 0.7164
Epoch 2/20
74/74 [==============================] - 0s 320us/step - loss: 0.5276 - binary_accuracy: 0.9010
Epoch 3/20
74/74 [==============================] - 0s 333us/step - loss: 0.4155 - binary_accuracy: 0.8991
Epoch 4/20
74/74 [==============================] - 0s 376us/step - loss: 0.3379 - binary_accuracy: 0.9006
Epoch 5/20
74/74 [==============================] - 0s 358us/step - loss: 0.2773 - binary_accuracy: 0.9034
Epoch 6/20
74/74 [==============================] - 0s 363us/step - loss: 0.2590 - binary_accuracy: 0.9055
Epoch 7/20
74/74 [==============================] - 0s 357us/step - loss: 0.2505 - binary_accuracy: 0.9084
Epoch 8/20
74/74 [==============================] - 0s 361us/step - loss: 0.2640 - binary_accuracy: 0.9032
Epoch 9/20
74/74 [==============================] - 0s 363us/step - loss: 0.2544 - binary_accuracy: 0.8989
Epoch 10/20
74/74 [==================

In [162]:
model.evaluate(x_test,y_test,batch_size=50)

32/32 [==============================] - 0s 336us/step - loss: 0.2544 - binary_accuracy: 0.8973


[0.25443828105926514, 0.8972904682159424]

# 정량적 분석 결과
## Deep Learning 모델을 사용한 모델이 조금 더 좋은 성능을 보임
- ## log loss 기준으로 평가
- ## DL Model : 0.27 
- ## PCA : 0.32

In [163]:
pred = model.predict(x_test)

In [164]:
pred[:20]

array([[0.02318577],
       [0.02193565],
       [0.00292798],
       [0.24914448],
       [0.08111358],
       [0.01729449],
       [0.08040125],
       [0.01359381],
       [0.02571154],
       [0.01988917],
       [0.46732932],
       [0.12309042],
       [0.05040107],
       [0.17365862],
       [0.04724884],
       [0.11495193],
       [0.09214848],
       [0.0037079 ],
       [0.2967868 ],
       [0.02597189]], dtype=float32)

In [165]:
y_test[:20]

3700    0
3701    0
3702    0
3703    0
3704    1
3705    0
3706    0
3707    0
3708    0
3709    0
3710    0
3711    0
3712    0
3713    0
3714    0
3715    0
3716    0
3717    0
3718    0
3719    0
Name: troll, dtype: int64

In [166]:
list_pred = []
for p in pred :
    list_pred.append(p[0])

series_pred = pd.Series(list_pred)
y_test.index = [i for i in range(train_data_len,train_data_len+len(y_test))]
series_pred.index = [i for i in range(train_data_len,train_data_len+len(y_test))]

In [167]:
df_shuffled_total

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
0,-0.434277,-0.618908,-0.113804,-0.705037,-0.705106,0,12,Qo3se6alXKmehQxgoyZ6YQA0mseJsPKEFc0us95ADoCxT2w,3860,3193,0,1029,3047,1011
1,0.307922,-0.152650,-0.829000,0.160268,0.164253,1,39,SAWxS77FXs03xyoW-WHErG8yMsWcwUrQDeENo0ZhLzOAR1w,1006,1006,1006,1006,1006,1006
2,-0.395869,-0.550199,0.003021,-0.525970,-0.950440,1,79,vjyd9sefqzumlcNDTyTxZJzwew09f1PbSsoa8xkojNezGlY,1004,1004,1004,1004,1004,1004
3,0.003808,-0.513583,-0.492969,-0.839031,-0.667064,0,43,yl5WkGJro85d4aDo_eOJ6n_5mY0W1GJe_PL53DdpJsmVxqI,3117,3174,3504,3853,0,0
4,1.609645,0.382305,-1.451541,2.347640,1.019909,0,350,qUI-X0G9B2KKO0B36UvsV3k2ju3vpVGM7l1cbkXLmrcnZZI,3030,3801,3853,2055,3114,3041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,-0.950406,-0.745653,0.060120,-0.766399,0.272138,0,28,wyw5P9-Xx8Lt9Irms24hGOoTD806zuzbSTwqf_m2jbt8LrE,1402,1082,2055,3108,3020,0
5283,-0.967382,-1.212238,-0.512353,-1.138467,-1.535250,1,20,kW9Fg7SpHhVH2DTxf2alC8yx2Ks4sRipqJtaSIkj5CoVGN0,1006,1006,1006,1006,1006,1006
5284,-0.271795,-0.897357,0.186923,-0.773690,-0.634453,0,245,FT7DGCuxdaLNkeTHWfKpBNnAnJErnnd0efR8gb2j15bnNn8,3020,3152,0,2055,1402,0
5285,0.191891,3.128101,0.044998,1.246856,-0.248878,0,202,4Nnf1APlZHkjuDnksXDxuty3tN5enJoDvsqlX2VRvFWf-A,1055,3009,3095,3031,3094,3046


In [168]:
from IPython.core.display import HTML

# df_shuffled_image = df_shuffled_total[train_data_len:][(y_test==1) & (series_pred<0.5)][:30].drop(['summoner_id','troll_count'],axis=1)

df_shuffled_image = df_shuffled_total[train_data_len:][(y_test==1) & (series_pred<0.5)][:30].drop(['summoner_id'],axis=1)
df_shuffled_image

for item_num in tqdm(itemStr) :
    df_shuffled_image[item_num] = df_shuffled_image[item_num].astype(str)
    df_shuffled_image[item_num] = '<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/' + df_shuffled_image[item_num] + '.png\' weight=25 height=25>'

for item_num in tqdm(itemStr) :
    for i,item_img in enumerate(df_shuffled_image[item_num]) :
        if item_img == "<img src = \' http://ddragon.leagueoflegends.com/cdn/10.18.1/img/item/0.png\' weight=25 height=25>" :
            df_shuffled_image[item_num][df_shuffled_image.index[i]] = ''

df_shuffled_image_html=df_shuffled_image.to_html(escape=False)
#print(df_shuffled_image_html[:1600])
# HTML(df_shuffled_image_html)

100%|██████████| 6/6 [00:00<00:00, 2144.33it/s]


In [169]:
df_shuffled_total[train_data_len:][pred>=0.5]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5
3854,-1.461604,-1.499141,0.540503,-2.691729,-1.927858,0,126,VHvSHIXeBsg_e6CUL9UK5P72SLmJqncsyiHG3bDqsVR2CIg,2031,1029,1055,1055,1029,2422
3881,-1.755942,-0.863269,0.311450,-2.777495,-0.834756,0,238,f_7GBGkZ_aXoP98657AGmO24uKozhiXj75nO3y_HBrXy8Rs,1006,0,3009,1006,1006,1006
4071,-1.601427,-1.390076,0.195787,-2.533516,-0.389738,1,84,AGV-4Uw0lGxIDtLHLIpvthe7NrXNN-P-LMgkuLgQ4yXBa90,1006,1006,1006,1006,1006,1006
4189,-0.659186,-1.683863,1.377592,-2.125983,-1.655485,0,51,Zqmi03u7YSHfn2_sRD9sUwqFCNQ6TPblGI9hF29tqxHrbjM,1054,2031,1038,1037,3006,1018
4234,-0.977179,-1.480054,1.071828,-2.423396,-1.831999,1,67,WKqW_J4cP92-OxKLxeHBbfYZoFIqsCBGBwueHhIbXXDcOl90,1006,1006,1006,1006,1006,1006
4262,-0.912081,-1.426838,-0.559579,-2.370317,-1.597007,1,157,AyXMFgxRRIugbGec4y2TkUonPTLY5LbCq8eZRgHE5adAx1o,1006,1006,1006,1006,1006,1006
4345,-1.376245,-1.426838,-1.317573,-2.428959,-1.867492,1,157,mOsGKBOaxsQQx0EPiFJxQxHOu2f6I4iJgRp1dTUCfh6k4Ahg,1006,1006,1006,1006,1006,1006
4358,-1.488203,-1.599778,1.244968,-2.445106,-1.835575,0,245,aoJgV7-d5C91XuPTuI1ax8A9ou8nmNE_U4Z4cWeJ16XW-BQ,3706,2031,1027,1052,2422,0
4380,-1.306673,-1.602435,0.280412,-2.447266,-1.332552,0,4,EqyXdPBFh79Btjb325Hf_AMYeN8Y4jMuWtMaC4kxcxU6O5U,3010,2033,1082,1001,0,0
4449,-1.392273,-1.683863,-0.404286,-2.985741,-2.997826,1,51,YCJsL8YUDPZI0luE0AOuPquTVKGAp_-jg45xHOiw8CpH7hQ,2421,2421,2421,0,0,0


In [170]:
[(y_test==0)][0][:10]

3700     True
3701     True
3702     True
3703     True
3704    False
3705     True
3706     True
3707     True
3708     True
3709     True
Name: troll, dtype: bool

In [171]:
y_test

3700    0
3701    0
3702    0
3703    0
3704    1
       ..
5282    0
5283    1
5284    0
5285    0
5286    1
Name: troll, Length: 1587, dtype: int64

In [172]:
[(y_test==0) & (series_pred>=0.5)]

[3700    False
 3701    False
 3702    False
 3703    False
 3704    False
         ...  
 5282    False
 5283    False
 5284    False
 5285    False
 5286    False
 Length: 1587, dtype: bool]

In [173]:
troll_count = []
for summoner_id in df_shuffled_total['summoner_id'] :
    troll_count.append(  troll_player_dict.get(summoner_id) )
df_shuffled_total['troll_count'] = troll_count
df_shuffled_total.fillna(value=0,inplace=True)

In [174]:
df_shuffled_total[train_data_len:][(y_test==0) & (series_pred>=0.5)][20:30]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5,troll_count


In [175]:
df_shuffled_total[train_data_len:][(y_test==1) & (series_pred<0.5)][:20]

,kill,assist,death,gold,level,troll,championid,summoner_id,item0,item1,item2,item3,item4,item5,troll_count
3704,-0.944560,0.227156,2.503619,-0.930351,-0.850999,1,4,n0CvO2naiQUWLLy1KcxuKDHYHw5ARmTmOykZpI70pq2IaAY,1027,1027,1027,1027,1027,1027,1.0
3726,-0.238124,-1.243594,-0.857495,-0.244350,0.182129,1,57,0SVWxrhpRSsjgaR9h6ygZgfqpDD-GqBmMMhCVFmPZkOzfPY,1006,1006,1006,1006,1006,1006,2.0
3728,-0.848298,-1.137299,-0.974400,-0.494091,0.406575,1,236,EIZjUUsXJbQznvjnl0F9TtqlRNstoOzX6LJY1w9jVtHBq-E,1028,1028,1028,1028,1028,1028,2.0
3752,0.122062,-1.318167,-1.784438,-1.378351,-1.503842,1,24,NWo79RlotE_gb-nWlEkof6YCamQwbZWvn8bNgUTRLmMOxRs,1006,1006,1006,1006,1006,1006,1.0
3753,-0.733793,-0.826430,0.094569,-1.510107,-0.694017,1,84,8Ni3-hn-ZzrJd7MPM4iIQ2-c_o2I10Y6JU4mqHEXMOZcyYI,1006,1006,1006,1006,1006,1006,1.0
3767,-0.993893,-1.477819,1.882961,-1.369367,-0.601392,1,38,G9RPFeEBWVdFfFdkgyIx65GOhDz0wEYSIRhZBz6pqthW5g,1082,1082,1082,1082,1082,1082,1.0
3772,-0.783320,-0.718356,0.034070,-1.327422,-1.025715,1,236,B1MLWiP4VjZAc4TiWXR5t6ECMvn99TaIDIrqZNziW5VZxF4,0,0,2421,2421,2421,2421,1.0
3776,-1.470916,-0.302079,-1.663509,-1.527761,0.683777,1,142,WMbBCQsCRHS69CmsGBc25e2eFuNcX2a9fyHApd6DHwmWL60,1006,1006,1006,1006,1006,1006,1.0
3800,-1.419465,-1.258827,0.165984,-1.974348,-1.412447,1,21,DpIFUrwm9YL-y5YMjheUMKrmIlrS_snhqCLM6NZonbmfRiE,1082,1082,1082,1082,1082,1082,1.0
3805,2.157112,-1.601696,-0.412435,1.443883,0.854493,1,142,iSxHw1hCDHlu22Hoct_B1-e58BFms78qyoVeTZizh3AdrUiU,2421,2421,2421,2421,2421,2421,1.0
